# Query Simbad on CALSPEC sources
============================================

- author Sylvie Dagoret-Campagne
- Affiliation LAL-IN2P3-CNRS
- Work : LSST & StarDice
- date May 4th 2017

In [1]:
import numpy as np
# Set up matplotlib and use a nicer set of plot parameters
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import pandas as pd
import os
import re

In [3]:
# to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [4]:
os.environ['PYSYN_CDBS']

'/Users/dagoret-campagnesylvie/MacOsX/External/PySyn_CDBS/grp/hst/cdbs'

In [5]:
from astropy.io import fits
import pysynphot as S
S.primary_area=6*1e4
S.binning=10.

# Query Simbad:
https://astroquery.readthedocs.io/en/latest/simbad/simbad.html

In [13]:
from astroquery.simbad import Simbad

In [6]:
SEDfile_dir=os.path.join(os.environ['PYSYN_CDBS'], 'calspec')

In [7]:
filelist=os.listdir(SEDfile_dir) 

## Extract the fits file headers

In [9]:
star_header = []
for filename in filelist:
    index=0
    if re.search('fits',filename):  #example of filename filter
        index+=1
        fullfilename = os.path.join(os.environ['PYSYN_CDBS'], 'calspec',filename)
        hdr = fits.getheader(fullfilename)
        star_header.append(hdr)

    START STIS                                                                   [astropy.io.fits.card]
FILE WRITTEN BY ECHLREDUCE.PRO ON 11-Jun-2013 11:27:39.00                        [astropy.io.fits.card]
coadd list for E*H:                                                              [astropy.io.fits.card]
 o57u01030 obb001040 obb005010 obb005020 obb001010 obb001090 obb005030           [astropy.io.fits.card]
 obb005040 obb001060 obb001030 o57u01020 obb001050 obb001020 obb0010a0           [astropy.io.fits.card]
 obb001070 o6hb100f0 obb001080 o57u01040 o5i011010 o5i011020 o5i011030           [astropy.io.fits.card]
 obb0010b0 obb002090 obb0020b0 obb002010 obb002030 o6hb20060 obb002020           [astropy.io.fits.card]
 o6hb20070 obb002040 o6hb20080 obb002050 obb002060 o6hb20090 obb002070           [astropy.io.fits.card]
 obb002080 obb0020a0 obb0020c0 obb0020d0 obb004080 obb053040 obb053080           [astropy.io.fits.card]
 obb053010 obb053020 obb053030 obb053050 obb053060 obb0530a0 obb

In [10]:
star_header[1][7] # TARGETID

'10LAC'

## Extract the object name from the headers

In [11]:
star_names = []
index=0
for hdr in star_header:
    
    star_name=star_header[index][7]
    star_names.append(star_name)
    index+=1

In [12]:
star_names

['10LAC',
 '10LAC',
 '10LAC',
 '1732526 with NICMOS observations',
 '1732526',
 '1732526',
 '1732526',
 '1740346 with NICMOS observations',
 '1740346 with NICMOS observations',
 '1740346',
 '1740346',
 '1740346',
 '1743045 with NICMOS observations',
 '1743045 with NICMOS observations',
 '1743045',
 '1743045',
 '1743045',
 '1757132',
 '1757132',
 '1757132',
 '1802271 with NICMOS observations',
 '1802271',
 '1802271',
 '1802271',
 '1805292 with NICMOS observations',
 '1805292 with NICMOS observations',
 '1805292',
 '1805292',
 '1805292',
 '1808347',
 '1808347',
 '1808347',
 '1812095 with NICMOS observations',
 '1812095 with NICMOS observations',
 '1812095',
 '1812095',
 '1812095',
 '1812524 with NICMOS observations',
 '1812524',
 '1812524',
 '2M0036+18 with STIS & NICMOS observations',
 '2M0036+18 with STIS & NICMOS observations',
 'Standard star flux with re-calibrated STIS & NICMOS observations---',
 '2M0036+18',
 '2M0036+18',
 '2M0036+18',
 '2M0559-14 with STIS & NICMOS observations',

In [75]:
class CalibSpecStar:
    """
    Class for the CALSPEC stars
    """
    name = ""          # the name of the star from CALSPEC/Pysynphot
    simbadname=""      # the name of the star returned by Astroquery-Simbad 
    RA =""             # string Right assension
    DEC=""             # string declination

    def __init__(self, name,simbname,ra,dec):   # constructor
        self.name=name
        self.simbname=simbname
        self.RA=ra
        self.DEC=dec
    def show(self):                             # print 
        print 'name=',self.name,', ra=',self.RA,', dec=',self.DEC
    def getName(self):                          # getters
        return self.name
    def getRA(self):
        return self.RA
    def getDEC(self):
        return self.DEC  

## Loop on entries to get info on CALSPEC

In [101]:
All_CALSPEC_Stars = {} #dictionnary of stars
count =0
for name in star_names: # loop on star names from Pysynphot-CALSPEC
    result_table = Simbad.query_object(name)   # Ask Simbad
    if result_table != None:                   #check if Simbad has found the star
        count+=1
        print "---------------------",count,') ',name,'-------------------------'
        print result_table
        # create an object with the found star
        newstar=CalibSpecStar(name,result_table["MAIN_ID"][0],result_table["RA"][0],result_table["DEC"][0])
        # fill the dictionnary only if there is no instance of that star already in dictionnary
        if not newstar.getName() in All_CALSPEC_Stars.keys():
            All_CALSPEC_Stars[newstar.getName()]= newstar # fill the directionary with the new star- object
            

--------------------- 1 )  10LAC -------------------------
 MAIN_ID        RA           DEC      ... COO_WAVELENGTH     COO_BIBCODE    
             "h:m:s"       "d:m:s"    ...                                   
--------- ------------- ------------- ... -------------- -------------------
*  10 Lac 22 39 15.6786 +39 03 00.971 ...              O 2007A&A...474..653V
--------------------- 2 )  10LAC -------------------------
 MAIN_ID        RA           DEC      ... COO_WAVELENGTH     COO_BIBCODE    
             "h:m:s"       "d:m:s"    ...                                   
--------- ------------- ------------- ... -------------- -------------------
*  10 Lac 22 39 15.6786 +39 03 00.971 ...              O 2007A&A...474..653V
--------------------- 3 )  10LAC -------------------------
 MAIN_ID        RA           DEC      ... COO_WAVELENGTH     COO_BIBCODE    
             "h:m:s"       "d:m:s"    ...                                   
--------- ------------- ------------- ... -----------

/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : NAME WD1026 453 with STIS observations
  (error.line, error.msg))


--------------------- 117 )  WD1057+719 -------------------------
  MAIN_ID        RA         DEC     ... COO_WAVELENGTH     COO_BIBCODE    
              "h:m:s"     "d:m:s"   ...                                   
----------- ----------- ----------- ... -------------- -------------------
PG 1057+719 11 00 34.24 +71 38 02.9 ...              I 2003yCat.2246....0C
--------------------- 118 )  WD1057+719 -------------------------
  MAIN_ID        RA         DEC     ... COO_WAVELENGTH     COO_BIBCODE    
              "h:m:s"     "d:m:s"   ...                                   
----------- ----------- ----------- ... -------------- -------------------
PG 1057+719 11 00 34.24 +71 38 02.9 ...              I 2003yCat.2246....0C
--------------------- 119 )  WD1057+719 -------------------------
  MAIN_ID        RA         DEC     ... COO_WAVELENGTH     COO_BIBCODE    
              "h:m:s"     "d:m:s"   ...                                   
----------- ----------- ----------- ... ------------

In [104]:
print " Number of CALSPEC stars recognised by Simbad : ",count

 Number of CALSPEC stars recognised by Simbad :  122


In [105]:
print All_CALSPEC_Stars.keys()

['10LAC', 'HD205905', 'HD14943', 'HD37725', 'HD160617', 'FEIGE34', 'HD159222', 'HZ43', 'HZ44', 'HZ21', 'WD1057+719', 'HZ4', 'FEIGE110', 'HD031128', 'GD71', 'HD209458', 'SIRIUS', 'WD0320-539', 'HD111980', 'HD009051', 'HD60753', 'LDS749B', 'HD180609', 'HD158485', 'HD37962', 'HD163466', 'HD185975', 'G191B2B', 'HD116405', 'ALPHA_LYR', 'HD200654', 'HZ43B', 'WD1657+343', 'HD165459', 'HD93521', 'GD153', 'WD0308-565', 'HD38949', 'VB8', 'HD106252', 'HD074000']
